In [1]:
import warnings
warnings.filterwarnings('ignore')

Download heart disease dataset heart.csv in [Exercise](https://github.com/codebasics/py/tree/master/ML/19_Bagging/Exercise) folder and do following, (credits of dataset:  https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1. Load heart disease dataset in pandas dataframe
2. Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3
3. Convert text columns to numbers using label encoding and one hot encoding
4. Apply scaling
5. Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.
6. Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance
7. Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.


[Solution Link](https://github.com/codebasics/py/blob/master/ML/19_Bagging/Exercise/bagging_heart_disease_prediction.ipynb)



 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df=pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [4]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


### Outlier treatment

In [5]:
# For RestingBP
lower_limit=df.RestingBP.mean()-df.RestingBP.std()
upper_limit=df.RestingBP.mean()+df.RestingBP.std()

In [6]:
# For Cholesterol
lower_limit1=df.Cholesterol.mean()-df.Cholesterol.std()
upper_limit1=df.Cholesterol.mean()+df.Cholesterol.std()

In [7]:
# For Oldpeak
lower_limit2=df.Oldpeak.mean()-df.Oldpeak.std()
upper_limit2=df.Oldpeak.mean()+df.Oldpeak.std()

In [9]:
df=df[((df.RestingBP>lower_limit) & (df.RestingBP<upper_limit)) & ((df.Cholesterol>lower_limit1) & (df.Cholesterol<upper_limit1)) & ((df.Oldpeak>lower_limit2) & (df.Oldpeak<upper_limit2))]

In [10]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0


In [11]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn.compose import ColumnTransformer

le=LabelEncoder()
df.Sex=le.fit_transform(df.Sex)

le1=LabelEncoder()
df.ChestPainType=le.fit_transform(df.ChestPainType)

le2=LabelEncoder()
df.RestingECG=le.fit_transform(df.RestingECG)

le3=LabelEncoder()
df.ExerciseAngina=le.fit_transform(df.ExerciseAngina)

le4=LabelEncoder()
df.ST_Slope=le.fit_transform(df.ST_Slope)

In [12]:
ct=ColumnTransformer(transformers=[('ChestPainType',OneHotEncoder(),['ChestPainType']),('RestingECG',OneHotEncoder(),['RestingECG'])],remainder='passthrough')

In [18]:
df1 =ct.fit_transform(df)
df1[0:3,:]

array([[  0. ,   1. ,   0. ,   0. ,   0. ,   1. ,   0. ,  40. ,   1. ,
        140. , 289. ,   0. , 172. ,   0. ,   0. ,   2. ,   0. ],
       [  0. ,   1. ,   0. ,   0. ,   0. ,   0. ,   1. ,  37. ,   1. ,
        130. , 283. ,   0. ,  98. ,   0. ,   0. ,   2. ,   0. ],
       [  1. ,   0. ,   0. ,   0. ,   0. ,   1. ,   0. ,  48. ,   0. ,
        138. , 214. ,   0. , 108. ,   1. ,   1.5,   1. ,   1. ]])

In [20]:
len(df1[0])

17

In [19]:
len(df1)

406

In [21]:
X=df1[:,:(len(df1[0])-1)]
y=df1[:,(len(df1[0])-1):]

In [22]:
y

array([[0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],

In [23]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(X,y)
svc.score(X,y)

0.6551724137931034

In [28]:
help(BaggingClassifier())

Help on BaggingClassifier in module sklearn.ensemble._bagging object:

class BaggingClassifier(sklearn.base.ClassifierMixin, BaseBagging)
 |  BaggingClassifier(base_estimator=None, n_estimators=10, *, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)
 |  
 |  A Bagging classifier.
 |  
 |  A Bagging classifier is an ensemble meta-estimator that fits base
 |  classifiers each on random subsets of the original dataset and then
 |  aggregate their individual predictions (either by voting or by averaging)
 |  to form a final prediction. Such a meta-estimator can typically be used as
 |  a way to reduce the variance of a black-box estimator (e.g., a decision
 |  tree), by introducing randomization into its construction procedure and
 |  then making an ensemble out of it.
 |  
 |  This algorithm encompasses several works from the literature. When random
 |  subsets of the dataset are draw

In [27]:
from sklearn.ensemble import BaggingClassifier
bg=BaggingClassifier(
    base_estimator=SVC(),
    n_estimators=100, 
    max_samples=1.0
)

In [29]:
bg.fit(X,y)
bg.score(X,y)

0.6551724137931034

In [30]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()

In [31]:
tree.fit(X,y)
tree.score(X,y)

1.0

In [33]:
bg1=BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100, 
    max_samples=1.0
)
bg1.fit(X,y)
bg1.score(X,y)

1.0